In [ ]:
import pandas as pd

# Carregar o dataset
dados = pd.read_csv("students_performance.csv")

# Verificar as primeiras linhas para entender os dados
print(dados.head())


In [4]:
import pandas as pd

# 1. Carregar o dataset
dados = pd.read_csv("students_performance.csv")

# 2. Verificar as colunas do dataset
print("Colunas disponíveis:", dados.columns)

# 3. Verificar se a coluna 'nota_final' existe no dataset
if 'nota_final' in dados.columns:
    # 4. Garantir que a coluna 'nota_final' está em formato numérico
    dados['nota_final'] = pd.to_numeric(dados['nota_final'], errors='coerce')

    # 5. Verificar se há valores ausentes (NaN) na coluna 'nota_final' e tratar
    if dados['nota_final'].isnull().sum() > 0:
        print(f"A coluna 'nota_final' contém {dados['nota_final'].isnull().sum()} valores ausentes.")
        # Preencher NaNs com a média da coluna 'nota_final'
        dados['nota_final'].fillna(dados['nota_final'].mean(), inplace=True)
        p


Colunas disponíveis: Index(['student_id', 'age', 'gender', 'parental_education', 'study_hours_week',
       'attendance_rate', 'extracurricular', 'sleep_hours', 'previous_scores',
       'tutoring', 'internet_quality', 'family_income', 'health_status',
       'final_grade', 'study_efficiency', 'sleep_quality'],
      dtype='object')


In [5]:
# Separar as variáveis independentes (features) e a variável dependente (target)
X = dados.drop('nota_final', axis=1)  # 'nota_final' é o target
y = dados['nota_final']


KeyError: "['nota_final'] not found in axis"

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados em treino (60%), validação (20%) e teste (20%)
X_temp, X_teste, y_temp, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
X_treino, X_validacao, y_treino, y_validacao = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Verificar o tamanho dos conjuntos
print(f"Tamanho treino: {len(X_treino)}")
print(f"Tamanho validação: {len(X_validacao)}")
print(f"Tamanho teste: {len(X_teste)}")


In [ ]:
from sklearn.linear_model import LinearRegression

# Criar o modelo de Regressão Linear
modelo = LinearRegression()

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)


In [ ]:
# Fazer previsões com os dados de treino e validação
prev_treino = modelo.predict(X_treino)
prev_val = modelo.predict(X_validacao)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Calcular R² para treino e validação
r2_treino = r2_score(y_treino, prev_treino)
r2_val = r2_score(y_validacao, prev_val)

# Calcular RMSE para treino e validação
rmse_treino = np.sqrt(mean_squared_error(y_treino, prev_treino))
rmse_val = np.sqrt(mean_squared_error(y_validacao, prev_val))

# Calcular MAE para treino e validação
mae_treino = mean_absolute_error(y_treino, prev_treino)
mae_val = mean_absolute_error(y_validacao, prev_val)

# Exibir as métricas
print(f"R² Treino: {r2_treino:.2f}, R² Validação: {r2_val:.2f}")
print(f"RMSE Treino: {rmse_treino:.2f}, RMSE Validação: {rmse_val:.2f}")
print(f"MAE Treino: {mae_treino:.2f}, MAE Validação: {mae_val:.2f}")


In [ ]:
# Comparar R² treino vs validação
diferenca_r2 = abs(r2_treino - r2_val)

if diferenca_r2 < 0.10:
    print("✅ Sem overfitting")
else:
    print("❌ Overfitting detectado")


In [ ]:
import matplotlib.pyplot as plt

# Calcular resíduos
residuos_treino = y_treino - prev_treino
residuos_val = y_validacao - prev_val

# Gráfico de resíduos para treino
plt.hist(residuos_treino, bins=30, edgecolor='black')
plt.title("Distribuição dos Resíduos - Treino")
plt.xlabel("Resíduo")
plt.ylabel("Frequência")
plt.show()

# Gráfico de resíduos para validação
plt.hist(residuos_val, bins=30, edgecolor='black')
plt.title("Distribuição dos Resíduos - Validação")
plt.xlabel("Resíduo")
plt.ylabel("Frequência")
plt.show()


In [ ]:
# Gráfico Predições vs Valores Reais (Validação)
plt.scatter(y_validacao, prev_val, color='blue', label='Predições')
plt.plot([min(y_validacao), max(y_validacao)], [min(y_validacao), max(y_validacao)], 'r--', label='Linha de Predição Perfeita')
plt.title("Predições vs Valores Reais - Validação")
plt.xlabel("Valor Real")
plt.ylabel("Valor Previsto")
plt.legend()
plt.show()


In [ ]:
import joblib

# Salvar o modelo
joblib.dump(modelo, 'modelo_baseline.pkl')
    